<a href="https://colab.research.google.com/github/DeepLabCut/DeepLabCut/blob/master/examples/COLAB/COLAB_transformer_reID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut 2.2 Toolbox Demo on how to use our Pose Transformer for unsupervised identity tracking of animals
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1628250004229-KVYD7JJVHYEFDJ32L9VJ/DLClogo2021.jpg?format=1000w)

https://github.com/DeepLabCut/DeepLabCut

### This notebook illustrates how to use the transformer for a multi-animal DeepLabCut (maDLC) Demo 3 mouse project:
- load our mini-demo data that includes a pretrained model and unlabeled video.
- analyze a novel video.
- use the transformer to do unsupervised ID tracking.
- create quality check plots and video.

### To create a full maDLC pipeline please see our full docs: https://deeplabcut.github.io/DeepLabCut/README.html
- Of interest is a full how-to for maDLC: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html
- a quick guide to maDLC: https://deeplabcut.github.io/DeepLabCut/docs/tutorial.html
- a demo COLAB for how to use maDLC on your own data: https://github.com/DeepLabCut/DeepLabCut/blob/main/examples/COLAB/COLAB_maDLC_TrainNetwork_VideoAnalysis.ipynb

### To get started, please go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [ ]:
# Install the latest DeepLabCut version:
!pip install "deeplabcut[tf]"

No information needs edited in the cells below, you can simply click run on each:

### Download our Demo Project from our server:

In [ ]:
# Download our demo project:
import requests
from io import BytesIO
from zipfile import ZipFile

url_record = 'https://zenodo.org/api/records/7883589'
response = requests.get(url_record)
if response.status_code == 200:
    file = response.json()['files'][0]
    title = file['key']
    print(f"Downloading {title}...")
    with requests.get(file['links']['self'], stream=True) as r:
        with ZipFile(BytesIO(r.content)) as zf:
            zf.extractall(path='/content')
else:
    raise ValueError(f'The URL {url_record} could not be reached.')

## Analyze a novel 3 mouse video with our maDLC DLCRNet, pretrained on 3 mice data 

###in one step, since auto_track=True you extract detections and association costs, create tracklets, & stitch them. We can use this to compare to the transformer-guided tracking below.


In [ ]:
import deeplabcut as dlc
import os

project_path = "/content/demo-me-2021-07-14"
config_path = os.path.join(project_path, "config.yaml")
video = os.path.join(project_path, "videos", "videocompressed1.mp4")

dlc.analyze_videos(config_path,[video], shuffle=0, videotype="mp4",auto_track=True)

### Next, you compute the local, spatio-temporal grouping and track body part assemblies frame-by-frame:

## Create a pretty video output:

In [ ]:
#Filter the predictions to remove small jitter, if desired:
dlc.filterpredictions(config_path,
                                 [video],
                                 shuffle=0,
                                 videotype='mp4',
                                 )

dlc.create_labeled_video(
    config_path,
    [video],
    videotype='mp4',
    shuffle=0,
    color_by="individual",
    keypoints_only=False,
    draw_skeleton=True,
    filtered=True,
)

Now, on the left panel if you click the folder icon, you will see the project folder "demo-me.."; click on this and go into "videos" and you can find the "..._id_labeled.mp4" video, which you can double-click on to download and inspect!

### Create Plots of your data:

> after running, you can look in "videos", "plot-poses" to check out the trajectories! (sometimes you need to click the folder refresh icon to see it). Within the folder, for example, see plotmus1.png to vide the bodyparts over time vs. pixel position.



In [ ]:
dlc.plot_trajectories(config_path, [video], shuffle=0,videotype='mp4')

# Transformer for reID

while the tracking here is very good without using the transformer, we want to demo the workflow for you! 

In [ ]:
dlc.transformer_reID(config_path, [video],
                     shuffle=0, videotype='mp4',
                     track_method='ellipse',n_triplets=100
                     )

now we can make another video with the transformer-guided tracking:


In [ ]:
dlc.plot_trajectories(config_path, [video],
                      shuffle=0,videotype='mp4',
                      track_method="transformer"
                      )

In [ ]:
dlc.create_labeled_video(
    config_path,
    [video],
    videotype='mp4',
    shuffle=0,
    color_by="individual",
    keypoints_only=False,
    draw_skeleton=True,
    track_method="transformer"
)